## Filter read alignments in bam file based on percent id with md value indicating percent id...

In [33]:
import pysam
from __future__ import division

In [2]:
test_md = '103^T53C2T341'

def _match_len(md):
    length=0
    mismatch=0
    number=""
    for i, c in enumerate(md):
        try:
            val = int(c)
            number = number+c
        except:
            if len(number) > 0:
                length += int(number)
                number=""
        if i == len(md)-1:
            length += int(number)
    return length

assert _match_len(test_md) == 499

In [3]:


md = lambda b: str(b).split("\t")[-1].split(",")[3].replace(")","").replace("'","").strip()

def filter_bam(bam, outbam, pctid = 95):
    with pysam.AlignmentFile(bam, "rb") as ih, pysam.AlignmentFile(outbam, "wb", template=ih) as oh:
        good = 0
        total = 0
        for i, l in enumerate(ih):
            if l.is_duplicate:
                continue

            total += 1
            md = str(l).split("\t")[-1].split(",")[3].replace(")","").replace("'","").strip()  #get md value from raw bam entry
            match = match_len(md)
            pct_match = (match)/l.rlen * 100

            if pct_match > pctid:
                good += 1
                oh.write(l)
        print("there were %s good read alignments out of %s total alignments" % (good, total))

In [4]:
bam = "../data/big_data/bam_exe/SRS019030_454_vs_OlaneusYIT12061.bam"

In [34]:
def read_overlap_pctid(samline, overlap, pctid):
    reallen = l.infer_query_length()
    alnlen = l.query_length
    mismatch = l.get_tag("NM")
    
    aln_overlap = alnlen/reallen * 100
    aln_pctid = (alnlen-mismatch)/alnlen * 100
    if aln_overlap >= overlap and aln_pctid >= pctid:
        return True
    else:
        return False

In [36]:
def test_read_overlap_pctid():
    a = pysam.AlignedSegment()
    a.query_name = "read_12345"
    a.query_sequence = "ACGT" * 10
    a.flag = 0
    a.reference_id = 0
    a.reference_start = 20
    a.mapping_quality = 20
    a.cigartuples = ((0, 10), (2, 1), (0, 9), (1, 1), (0, 20))
    a.next_reference_id = 0
    a.next_reference_start = 200
    a.template_length = 167
    a.query_qualities = pysam.qualitystring_to_array("1234") * 10
    a.set_tag("NM", 5)
    
    assert read_overlap_pctid(a, 95, 95) == True
    assert read_overlap_pctid(a, 100, 100) == False

100.0
98.997995992
100.0
98.997995992


In [35]:
from __future__ import division
with pysam.AlignmentFile(bam, "rb") as ih: #, pysam.AlignmentFile(outbam, "wb", template=ih) as oh:
    good = 0
    total = 0
    for i, l in enumerate(ih):  
        reallen = l.infer_query_length()
        alnlen = l.query_length
        mismatch = l.get_tag("NM")

        aln_overlap = alnlen/reallen * 100
        print("overlap:", aln_overlap)
        print(mismatch)
        print(alnlen)
        aln_pctid = (alnlen-mismatch)/alnlen * 100
        print("pctid:", aln_pctid)
        
        print(read_overlap_pctid(l, 95, 95))
        if i == 10:
            break

('overlap:', 100.0)
5
277
('pctid:', 98.19494584837545)
100.0
98.1949458484
True
('overlap:', 100.0)
5
459
('pctid:', 98.91067538126362)
100.0
98.9106753813
True
('overlap:', 100.0)
5
432
('pctid:', 98.8425925925926)
100.0
98.8425925926
True
('overlap:', 100.0)
5
480
('pctid:', 98.95833333333334)
100.0
98.9583333333
True
('overlap:', 100.0)
5
462
('pctid:', 98.91774891774891)
100.0
98.9177489177
True
('overlap:', 100.0)
5
375
('pctid:', 98.66666666666667)
100.0
98.6666666667
True
('overlap:', 100.0)
5
366
('pctid:', 98.63387978142076)
100.0
98.6338797814
True
('overlap:', 100.0)
5
446
('pctid:', 98.87892376681614)
100.0
98.8789237668
True
('overlap:', 100.0)
5
495
('pctid:', 98.98989898989899)
100.0
98.9898989899
True
('overlap:', 100.0)
5
528
('pctid:', 99.0530303030303)
100.0
99.053030303
True
('overlap:', 100.0)
5
499
('pctid:', 98.99799599198397)
100.0
98.997995992
True


Object `l.get_cigar_stats` not found.
